In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
import statsmodels.stats.multicomp as multicomp

In [2]:
data_df = pd.read_excel('../data/42255_2025_1311_MOESM5_ESM.xlsx', sheet_name='2H')

In [3]:
data_df

,ND,HFD,HFD+SAL,HFD+SANA
0,1.60,3.3,1.4,1.8
1,1.40,3.1,2.1,1.5
2,1.70,2.5,3.2,1.8
3,1.40,1.9,1.3,1.5
4,1.50,1.3,1.1,1.1
5,1.00,3.2,2.3,1.1
6,1.80,1.4,2.8,1.5
7,1.10,2.8,1.2,1.2
8,1.30,3.9,1.3,1.5
9,1.40,1.5,2.2,1.3


In [4]:
data_df.columns

Index(['ND', 'HFD', 'HFD+SAL ', 'HFD+SANA'], dtype='object')

In [5]:
grp1, grp2, grp3, grp4 = data_df['ND'].dropna().values, \
data_df['HFD'].dropna().values, \
data_df['HFD+SAL '].dropna().values, \
data_df['HFD+SANA'].dropna().values

In [6]:
collect = [grp1, grp2, grp3, grp4]

In [7]:
def reorder_label(label):
    if label[-2:] == 'ND':
        return 'ND vs ' + label.split('vs')[0].strip()
    else:
        return label

In [8]:
for _ in [0]:

    print('N samples: ' + str(len(np.concatenate(collect))))
    print('ANOVA:')
    f = scipy.stats.f_oneway(grp1, grp2, grp3, grp4)
    print('F(3, ' + str(len(np.concatenate(collect))-4) + ') = ' + str(f.statistic) + ', p = ' + str(f.pvalue))

    #print('Bonferroni:')
    comp = multicomp.MultiComparison(data=np.concatenate(collect), 
                                      groups=np.concatenate([['ND']*len(grp1), 
                                                             ['HFD']*len(grp2), 
                                                             ['HFD+SAL']*len(grp3), 
                                                             ['HFD+SANA']*len(grp4)]))
    tbl, a1, a2 = comp.allpairtest(scipy.stats.ttest_ind, method= "bonf", alpha=0.05)
    bonf = pd.DataFrame(tbl)
    bonf.columns = bonf.loc[0].astype(str)
    bonf = bonf[1:]
    bonf['pval_corr'] = a1[2]
    bonf['g1'] = bonf['group1']
    bonf['g2'] = bonf['group2']
    bonf_df = bonf.copy()
    bonf_df['Bonferroni p-value'] = bonf_df['pval_corr']
    bonf_df['Comparison'] = bonf_df['g1'].astype(str) + ' vs ' + bonf_df['g2'].astype(str)
    bonf_df['Comparison'] = bonf_df['Comparison'].apply(reorder_label)

    #print('Tukey:')
    tukey = scipy.stats.tukey_hsd(*collect).pvalue
    tukey_df = pd.DataFrame({'g1':['ND', 'ND', 'ND', 'HFD', 'HFD', 'HFD+SAL'], 
     'g2':['HFD', 'HFD+SAL', 'HFD+SANA', 'HFD+SAL', 'HFD+SANA', 'HFD+SANA'],
     'Tukey p-value':[tukey[0][1], tukey[0][2], tukey[0][3], tukey[1][2], tukey[1][3], tukey[2][3]]})
    tukey_df['Comparison'] = tukey_df['g1'].astype(str) + ' vs ' + tukey_df['g2'].astype(str)

    combo_df = pd.merge(bonf_df, tukey_df, on=['Comparison'], how='outer')
    combo_df['Published p-value'] = ''
    combo_df = combo_df[['Comparison', 'Published p-value', 'Bonferroni p-value', 'Tukey p-value']].set_index('Comparison')
    print(combo_df.to_markdown())

N samples: 60
ANOVA:
F(3, 56) = 8.890889670333198, p = 6.51962938481444e-05
| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |                     |           0.618207   |     0.111174    |
| HFD vs HFD+SANA     |                     |           0.00147476 |     0.000243877 |
| ND vs HFD           |                     |           0.00104684 |     0.000170499 |
| HFD+SAL vs HFD+SANA |                     |           0.345884   |     0.258984    |
| ND vs HFD+SAL       |                     |           0.266737   |     0.21712     |
| ND vs HFD+SANA      |                     |           1          |     0.999512    |


Here is Figure 2H:

![file](../img/pub_fig_2h.png)

This is described as involving a one-way ANOVA followed by Bonferroni post-hoc. Using the [source data](https://www.nature.com/articles/s42255-025-01311-z#Sec38), we arrive at F(3, 56) = 8.89, p = 6.52E-5. However, most of the p-values shown for the post-hoc test are incorrect both for Bonferroni post-hoc comparison (i.e., independent t-tests between groups and Bonferroni FWER correction on p-values for each pairwise test) and for Tukey's post-hoc test. See summary table below.

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |             0.11        |           0.618   |     0.111    |
| HFD vs HFD+SANA     |                0.0008     |           0.00147 |     0.000244 |
| ND vs HFD           |              0.0007       |           0.00105 |     0.000170 |
| HFD+SAL vs HFD+SANA |             0.26        |           0.346   |     0.259    |
| ND vs HFD+SAL       |           Not shown          |           0.267   |     0.217     |
| ND vs HFD+SANA      |             0.997        |           1.00          |     1.00    |

Could the authors clarify? 

The code for this analysis is available at [github.com/reeserich/cal_et_al_2025](https://github.com/reeserich/cal_et_al_2025).